# SVD를 이용한 추천시스템

- 💬 REMIND 
    - Memory 기반 추천 시스템
        - 추천을 위한 모든 데이터를 메모리에 저장, 추천이 필요할 때마다 이를 사용
        - 개별 사용자 데이터에 집중
        - 대량의 데이터 처리 문제 (실시간 추천 불가)
        - Cold start 문제 : 새로운 사용자, 아이템이 시스템에 추가될 때 추천이 불가한 문제
    - Model 기반 추천 시스템
        - 데이터로부터 추천을 위한 모델(기계학습)을 구성, 이 모델을 사용하여 추천
        - 학습 과정에서 모델이 데이터의 특징을 배워서 데이터 정보를 압축
        - 데이터의 pattern을 학습 (유사도 보다 **특징의 요소를 더 자세히 반영**)
        - 데이터의 크기, 특징을 **동적** 으로 활용
        - 데이터의 **잠재적 특성**을 파악하는 모델 (latent factor model)
            - 장점
                - 추천된 모델은 압축된 형태로 저장(저차원의 잠재적 특성으로 표현)
                - 학습 과정 시간은 오래 걸리나 모델이 만들어진 후 예측 시간은 빠름
                - 과적합 방지 : 데이터를 다양하게 학습, 새로운 추천 가능성 높음

### 협업 필터링 - Model Based Approach
- 협업 필터링 - model based approach의 단점
    1. 데이터가 축적 X or Sparse 한 경우 성능이 낮음
    2. 확장 가능성이 낮음 (데이터가 너무 많아지면, 속도가 저하됨)
- 극복 방안
    - `기계 학습 알고리즘을 통해, 사용자가 아직 평가하지 않은 아이템의 평점을 예측`
    - 기본 아이디어 : 사용자의 선호도는 소수의 Hidden Factor로 결정 가능
        - Hidden Factor (숨겨진 요인 , = Embedding) : 사용자와 아이템의 특성을 저차원 공간에서 표현하는 잠재 변수를 의미
        - Embedding 이란? : 아이템과 유저에 대한 압축된 저차원 Hidden Factor
    - `행렬 분해 (Matrix Factorization : MF)` : 이러한 잠재 요인을 추출하는 대표적인 기법
        - 행렬 분해는 Loss Function과 제약 조건 (Constraints) 이 있는 최적화 문제로 공식화됨
    - 비모수적 (Non-Parametric) 방법
    - 딥러닝 (Deep Learning) 기반 방법
    


#### Summary
- Latent Factor Model
    - 사용자 - item 특성 간 복잡한 관계를 학습
    - 사용자 - item 행렬에서 사용자와 item을 factor(특징)로 나타내는 방법
    - 사용자와 item을 같은 vector공간(저차원)에 표현
    - Vector 공간 상에서의 거리로 유사도 평가
- 알고리듬
    - Matrix Factorization(행렬분해)
        - SVD
        - MF (명시적 평가값), NMF (비음수), IMF(암묵적 평가값), BPR (암묵적 평가값)
    - Factorization Machines (평가값, 속성정보)
    - Probabilistic Models : Clustering, Bayes Rules
    - Regression methods

## Env

In [59]:
import scipy
import numpy as np
import pandas as pd

## Data load

In [60]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.user', sep='|', names=u_cols, encoding='latin-1')

In [61]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown',
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary',
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western']
movies = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [62]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [63]:
# timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


## sparse matrix 밀도 check
- 추천시스템에서 사용자-아이템 평점 데이터의 특성을 이해하고, 이후 작업(모델 선택, 데이터 전처리 등)에 반영하기 위함
    - 사용자-아이템 평점 데이터는 일반적으로 매우 희소한 데이터로 구성
    - 밀도 (density) : 전체 데이터 중 실제로 값이 채워진 비율(관측된 평점 비율), 데이터의 희소성을 정량적으로 평가하는 지표
    - 데이터가 희소하거나 밀도가 낮을 경우, 일부 추천 알고리즘은 성능이 저하 가능
    

In [64]:
rating_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')

In [65]:
# 희소 정보
user_num = len(rating_matrix.index)
item_num = len(rating_matrix.columns)
non_null_num = user_num*item_num - rating_matrix.isnull().sum().sum()
non_null_ratio = non_null_num / (user_num*item_num)

print(f'사용자 수={user_num}, 아이템 수={item_num}, 밀도={non_null_ratio:.2f}')

사용자 수=943, 아이템 수=1682, 밀도=0.06


- 전체 가능한 $943 X 1682$ 조합 중 6% 만 평점이 채워져 있음을 의미

## SVD
- 행렬 분해 기반 모델 (SVD)은 희소한 데이터에서도 상대적으로 성능이 괜찮다.

In [66]:
# rating_matrix의 결손값은 0으로 채운다
matrix = rating_matrix.fillna(0).to_numpy()

# 인자 수 x 특이값 분해를 수행
P, S, Qt = scipy.sparse.linalg.svds(matrix, k=5) # 상위 5개의 특이값과 이에 해당하는 벡터만 사용하여 차원 축소

# 예측 평갓값 행렬 특이값 5개 만을 이용한 영화 평점
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)

print(f"P: {P.shape}, S: {S.shape}, Qt: {Qt.shape}, pred_matrix: {pred_matrix.shape}")
print('Sigma Value:\n',np.round(S, 4))


P: (943, 5), S: (5,), Qt: (5, 1682), pred_matrix: (943, 1682)
Sigma Value:
 [158.2119 159.1536 217.8462 244.8363 640.6336]


- $P$ : 사용자-잠재요인 행렬 (943 X 5)
- $S$ : 특이값 행렬 (5, )
- $Q^T$ : 아이템-잠재요인 행렬 (5 X 1682)
- $pred_matrix$ : 근사행렬을 재구성하여 **모든 사용자-아이템 조합에 대한 예측 평점 행렬**을 생성
    - 실제로 관측되지 않은 0 평점 (결측값 포함)에 대해서도 예측값을 제공
- 특이값 : $R$은 중요한 정보를 나타내며, 값이 클수록 해당 축이 데이터의 변동을 많이 설명

#### 💭 특이값 (Singular Value)의 의미
1. 데이터의 변동을 설명
    - 사용자 - 아이템 평점 행렬에서 가장 중요한 패턴(구조)를 나타냄
    - 각 특이값은 데이터의 특정 차원(축)이 얼마나 많은 변동(정보)을 설명하는지 나타냄
    - 값이 클수록 해당 차원이 데이터에서 더 중요한 정보를 포함하고 있다는 뜻
2. 중요한 정보의 우선순위
    - 특이값들은 내림차순 정렬, 가장 큰 특이값은 데이터의 변동을 가장 많이 설명하는 축을 나타냄

## movie data 재구성: [movie_id, title]

In [67]:
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [68]:
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

## RMSE

In [69]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

## 모델별로 테스트데이터의 예측 및 실데이터 간의 정확도 계산

In [70]:
# 모델별 RMSE를 계산하는 함수
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

## train, test set 분리
- user_id를 기준으로 일정 비율(stratify=true)로 학습, 테스트 데이터 분리

In [71]:
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state= 42, stratify=y)

## 학습데이터(사용자 X 영화 X 평점) matrix

In [72]:
# train 데이터로 Full matrix 구하기
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,NaN,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SVD 추천 함수


In [73]:
def SVD(user_id, movie_id):
    if movie_id in svd_rating_matrix.columns:
        svd_rating = svd_rating_matrix.loc[user_id][movie_id]
    else:
        svd_rating = 3.0
    return svd_rating

## test data에 대해 SVD결과와 실제 평가 결과 (RMSE)

In [74]:
matrix = rating_matrix.fillna(0).to_numpy()
#matrix = rating_matrix.fillna(rating_matrix.mean()).to_numpy()

# 인자 수 x 특이값 분해를 수행
P, S, Qt = scipy.sparse.linalg.svds(matrix, k=20)
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)

In [75]:
svd_rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.006481,1.809370,0.960381,2.481368,1.059846,0.546573,4.538669,2.297971,3.552747,1.490523,...,-0.001822,0.009433,-0.019013,0.040106,0.011884,-0.007336,-0.007700,-0.005133,0.010175,0.079008
2,1.894966,0.221923,-0.029699,0.090011,-0.001862,0.113968,0.867180,0.422012,1.245891,0.468456,...,0.001849,0.008298,0.009367,-0.012678,-0.006713,0.006210,0.010882,0.007254,0.008723,-0.022699
3,0.260253,-0.066518,0.146165,-0.129483,-0.048204,0.019336,0.009599,0.005938,-0.633178,0.066842,...,0.004879,0.006192,0.006147,-0.006796,0.009542,-0.006157,0.030835,0.020557,-0.001604,0.000979
4,0.016350,-0.096596,0.091923,-0.023741,0.216779,0.038250,-0.046003,-0.105071,-0.410620,-0.141353,...,0.002884,0.000804,0.006543,-0.003759,-0.000861,-0.005185,0.013143,0.008762,-0.001786,-0.006152
5,3.698721,1.201589,0.374176,1.407907,0.241429,0.036356,1.815734,1.443525,-0.421585,0.468069,...,0.001371,0.004985,-0.013011,-0.012842,-0.016411,-0.010507,-0.002918,-0.001945,0.007771,-0.002175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2.064812,-0.065772,0.290639,-0.175145,-0.113272,-0.019625,1.021411,0.096239,1.418665,0.153576,...,-0.001338,-0.005229,0.006062,0.001894,0.007092,0.014316,-0.008350,-0.005567,-0.008536,-0.003219
940,1.434438,0.090097,-0.087964,1.069583,0.043903,0.011418,1.351274,0.794164,1.084095,0.343653,...,0.003133,-0.000181,0.005936,0.015025,-0.006492,0.014885,0.020663,0.013775,-0.001134,-0.014706
941,1.436732,0.019490,0.505868,0.074938,-0.042586,-0.003322,1.490683,0.178519,0.598166,0.199237,...,-0.000710,0.004634,-0.004551,-0.002483,0.000896,0.011135,-0.006277,-0.004184,0.008948,0.003816


In [76]:
score(SVD) # 2.709285376137203

2.7092853761372027

In [ ]:
# 인자 수 x 특이값 분해를 수행
P, S, Qt = scipy.sparse.linalg.svds(matrix, k=11) # 최적의 Latent Factor 적용
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)

In [78]:
score(SVD) # 2.709285376137203

2.669090545779567

## rating matrix에서 Nan 값 사용자별 평점으로 한 것과 비교

- `matrix = rating_matrix.fillna(0).to_numpy()` : 결측값을 0 으로 채움
- `matrix = ratings_matrix.fillna(rating_matrix.mean(axis=1).to_numpy()` : 결측값을 사용자별 평균 평점으로 채움
  - NaN을 사용자별 평균으로 채우면, 개인화된 선호도가 반영

# 전체 데이터에서 추천
- 테스트 결과는 모델의 신뢰성을 검증하기 위한 도구
- 실제 추천은 전체 데이터를 대상으로 이루어짐
    - `추천 시스템의 최종 목표` : 전체 데이터에서 사용자가 아직 보지 않은 아이템을 추천하는 것

In [79]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

# 미평가한 영화 0점
#matrix = rating_matrix.fillna(0).to_numpy()

# 미평가한 영화 영화별 평점 평균
copy_rating_matrix= rating_matrix.fillna(rating_matrix.mean())
matrix = copy_rating_matrix.to_numpy()

# 인자 수 x 특이값 분해를 수행한다
P, S, Qt = scipy.sparse.linalg.svds(matrix, k=10)

# 예측 평갓값 행렬 특이값 5개 만을 이용한 영화 평점
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)

In [80]:
def recommender(user, n_items=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)   # user가 미평가한 영화
    for item in items.index:
        predictions.append(SVD(user, item))                   # 예상평점 계산
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

In [81]:
recommender(user=2, n_items=5)

movie_id
1467                                     Cure, The (1995)
1122                    Last Time I Saw Paris, The (1954)
1653                         Chairman of the Board (1998)
1293                     Ayn Rand: A Sense of Life (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
Name: title, dtype: object

In [82]:
recommender(user=20, n_items=5)

movie_id
1122              Last Time I Saw Paris, The (1954)
1201    Maybe, Maybe Not (Bewegte Mann, Der) (1994)
1467                               Cure, The (1995)
1599                            Guantanamera (1994)
814                             One Fine Day (1996)
Name: title, dtype: object

## 최적 latent factor 구하기
### 💭 Latent Factor란?
- SVD, 행렬 분해 등의 기법에서 **사용자와 아이템 간의 상호작용을 설명하는 잠재공간(Latent Space)**의 차원을 의미
- 사용자와 아이템은 이 잠재공간에서 각각 벡터로 표현, 벡터 간 내적을 통해 예측 평점을 계산
     - 예시 : 영화 추천에서 Latent Factor는 장르 선호도, 액션/로맨스 의 비율 등과 같은 숨겨진 요인을 나타낼 수 있음

- 최적의 Latent Factor 구하는 이유
    - Latent Factor 수는 모델 성능에 직접적인 영향을 미친다.
    - 너무 작으면 : 사용자 - 아이템 관계를 충분히 표현하지 못해 과소적합
    - 너무 크면 : 데이터의 노이즈 까지 학습하여 과적합 발생가능성 증가
    - `사용자와 아이템 간의 숨겨진 특징을 설명하기에 적절한 Latent Factor 수를 찾아야 함`

In [83]:
# train set으로 factor 갯수에 따른 RMSE
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
copy_rating_matrix= rating_matrix.fillna(rating_matrix.mean())
matrix = copy_rating_matrix.to_numpy()

for factor in [20, 50, 100, 150, 200, 250]:
    # 인자 수 x 특이값 분해를 수행한다
    P, S, Qt = scipy.sparse.linalg.svds(matrix, k=factor)

    # 예측 평갓값 행렬 특이값 5개 만을 이용한 영화 평점)
    pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
    svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)
    print("Neighbor size = %d : RMSE = %.4f" % (factor, score(SVD)))


Neighbor size = 20 : RMSE = 0.9888
Neighbor size = 50 : RMSE = 1.0000
Neighbor size = 100 : RMSE = 1.0112
Neighbor size = 150 : RMSE = 1.0183
Neighbor size = 200 : RMSE = 1.0232
Neighbor size = 250 : RMSE = 1.0257


-> Factor 개수가 20 일때, RMSE가 가장 낮은 값을 보이므로, 이 경우가 적절하다.
- RMSE가 증가하기 시작하면 과적합의 가능성이 증가하면서 성능이 저하됨 

In [84]:
# train set으로 factor 갯수에 따른 RMSE
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
copy_rating_matrix= rating_matrix.fillna(rating_matrix.mean())
matrix = copy_rating_matrix.to_numpy()

for factor in [2, 4, 6, 8, 10, 11, 12, 13, 14, 16, 18, 20]:
    # 인자 수 x 특이값 분해를 수행한다
    P, S, Qt = scipy.sparse.linalg.svds(matrix, k=factor)

    # 예측 평갓값 행렬 특이값 5개 만을 이용한 영화 평점)
    pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
    svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)
    print("Neighbor size = %d : RMSE = %.4f" % (factor, score(SVD)))


Neighbor size = 2 : RMSE = 1.0007
Neighbor size = 4 : RMSE = 0.9926
Neighbor size = 6 : RMSE = 0.9898
Neighbor size = 8 : RMSE = 0.9867
Neighbor size = 10 : RMSE = 0.9864
Neighbor size = 11 : RMSE = 0.9851
Neighbor size = 12 : RMSE = 0.9861
Neighbor size = 13 : RMSE = 0.9862
Neighbor size = 14 : RMSE = 0.9869
Neighbor size = 16 : RMSE = 0.9877
Neighbor size = 18 : RMSE = 0.9883
Neighbor size = 20 : RMSE = 0.9888


# 최적의 Latent Factor로 전체 데이터에서 추천

In [85]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

# 미평가한 영화 0점
#matrix = rating_matrix.fillna(0).to_numpy()

# 미평가한 영화 영화별 평점 평균
copy_rating_matrix= rating_matrix.fillna(rating_matrix.mean())
matrix = copy_rating_matrix.to_numpy()

# 인자 수 x 특이값 분해를 수행한다
P, S, Qt = scipy.sparse.linalg.svds(matrix, k=11)

# 예측 평갓값 행렬 특이값 5개 만을 이용한 영화 평점
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)
svd_rating_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)

In [86]:
recommender(user=2, n_items=5)

movie_id
1536                                    Cosi (1996)
1293               Ayn Rand: A Sense of Life (1997)
1201    Maybe, Maybe Not (Bewegte Mann, Der) (1994)
1189                        That Old Feeling (1997)
814                             One Fine Day (1996)
Name: title, dtype: object

In [87]:
recommender(user=20, n_items=5)

movie_id
1536                                    Cosi (1996)
1201    Maybe, Maybe Not (Bewegte Mann, Der) (1994)
1467                               Cure, The (1995)
1122              Last Time I Saw Paris, The (1954)
1653                   Chairman of the Board (1998)
Name: title, dtype: object

- RMSE가 최소인 Latent Factor는 모델의 성능이 최적화되었음을 나타낼 수 있지만,
- `이것이 항상 최적의 Latent Factor를 의미하지는 않는다`
    - 최적의 Latent Factor 판단 기준으로 일반화 성능, 평가 지표 다양화, Latent Factor의 해석 가능성, 복잡도와 효율성 등을 판단해야한다.